# FinBERT Profiling: Baseline 

This notebook is intentionally **thin**: it reuses the profiling utilities in `pipelines/finBERT/finbert/` (especially `finbert/finbert_profile.py` and `finbert/profile_utils.py`) instead of copying large code blocks.




In [20]:
from __future__ import annotations

from pathlib import Path
import shutil
import os
import logging
import sys
sys.path.append('..')

from textblob import TextBlob
from pprint import pprint
from sklearn.metrics import classification_report

from transformers import AutoModelForSequenceClassification

from finbert.finbert import *
from finbert.finbert_profile import *
from finbert.profile_utils import get_model_size_mb, print_device_info, setup_nltk_data
import finbert.utils as tools

from finbert.distillFinBert import DistillFinBert


%load_ext autoreload
%autoreload 2

project_dir = Path.cwd().parent
pd.set_option('max_colwidth', None)

import wandb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
wandb.init(
    entity="si2449-columbia-university",
    project="finbert-experiments",
    name="training-distilled_bert-profiled",
    group="knowledge-distillation",
)

In [22]:
cl_path = project_dir/'models'/'student'
cl_data_path = project_dir/'data'/'sentiment_data'

In [23]:
try:
    shutil.rmtree(cl_path) 
except:
    pass

bertmodel = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', cache_dir=None, num_labels=3)


config = Config(   data_dir=cl_data_path,
                   bert_model=bertmodel,
                   num_train_epochs=4,
                   model_dir=cl_path,
                   max_seq_length = 48,
                   train_batch_size = 32,
                   learning_rate = 2e-5,
                   output_mode='classification',
                   warm_up_proportion=0.2,
                   local_rank=-1,
                   discriminate=True,
                   gradual_unfreeze=True)

config.profile_train_steps = 20

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
from contextlib import nullcontext
from typing import Any

import torch.nn.functional as F
from sklearn.metrics import f1_score


def timed_eval(
    *, finbert: FinBert, model: torch.nn.Module, examples, use_amp: bool
) -> tuple[pd.DataFrame, dict[str, Any]]:
    """Evaluation loop with optional CUDA autocast + timing (kept small for notebook use)."""
    loader = finbert.get_loader(examples, phase="eval")
    device = finbert.device

    model.eval()
    preds: list[np.ndarray] = []
    labels: list[int] = []

    if device.type == "cuda":
        torch.cuda.synchronize(device)

    start = time.perf_counter()
    with torch.no_grad():
        for batch in loader:
            batch = tuple(t.to(device) for t in batch)
            input_ids, attention_mask, token_type_ids, label_ids, _agree_ids = batch

            logits = model(input_ids=input_ids, attention_mask=attention_mask)[0]

            preds.extend(logits.detach().cpu().numpy())
            labels.extend(label_ids.detach().cpu().numpy().tolist())

    if device.type == "cuda":
        torch.cuda.synchronize(device)

    wall_s = time.perf_counter() - start
    n = len(labels)

    results_df = pd.DataFrame({"predictions": preds, "labels": labels})

    timing = {
        "eval_wall_s": float(wall_s),
        "eval_num_samples": int(n),
        "eval_samples_per_s": float(n / wall_s) if wall_s > 0 else float("inf"),
    }
    return results_df, timing

In [25]:
finbert = DistillFinBert(config)
finbert.base_model = 'bert-base-uncased'
finbert.config.discriminate=True
finbert.config.gradual_unfreeze=True

In [26]:
finbert.prepare_model(label_list=['positive','negative','neutral'])

12/15/2025 14:25:06 - INFO - finbert.distillFinBert -   device: cuda n_gpu: 1, distributed training: False, 16-bits training: False


In [27]:
train_data = finbert.get_data('train')

In [28]:
model = finbert.create_the_model()

In [29]:
freeze = 6 

for param in model.distilbert.embeddings.parameters():
    param.requires_grad = False

for i in range(min(freeze, len(model.distilbert.transformer.layer))):
    for param in model.distilbert.transformer.layer[i].parameters():
        param.requires_grad = False

In [30]:
start = time.perf_counter()
trained_model = finbert.train(train_examples = train_data, model = model)
train_wall_s = time.perf_counter() - start

12/15/2025 14:25:08 - INFO - finbert.utils -   *** Example ***
12/15/2025 14:25:09 - INFO - finbert.utils -   guid: train-1
12/15/2025 14:25:09 - INFO - finbert.utils -   tokens: [CLS] after the reporting period , bio ##tie north american licensing partner so ##max ##on pharmaceuticals announced positive results with na ##lm ##efe ##ne in a pilot phase 2 clinical trial for smoking ce ##ssa ##tion [SEP]
12/15/2025 14:25:09 - INFO - finbert.utils -   input_ids: 101 2044 1996 7316 2558 1010 16012 9515 2167 2137 13202 4256 2061 17848 2239 24797 2623 3893 3463 2007 6583 13728 27235 2638 1999 1037 4405 4403 1016 6612 3979 2005 9422 8292 11488 3508 102 0 0 0 0 0 0 0 0 0 0 0
12/15/2025 14:25:09 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0
12/15/2025 14:25:09 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/15/

12/15/2025 14:25:09 - INFO - finbert.distillFinBert -   ***** Loading data *****
12/15/2025 14:25:09 - INFO - finbert.distillFinBert -     Num examples = 3488
12/15/2025 14:25:09 - INFO - finbert.distillFinBert -     Batch size = 32
12/15/2025 14:25:09 - INFO - finbert.distillFinBert -     Num steps = 48
/home/srm2245/hpml-project/pipelines/finBERT/notebooks/../finbert/distillFinBert.py:372: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=self.config.use_amp)
Epoch:   0%|          | 0/4 [00:00<?, ?it/s]


AttributeError: 'DistillFinBert' object has no attribute '_is_distilbert'

In [ ]:
test_data = finbert.get_data("test")

results = finbert.evaluate(examples=test_data, model=trained_model)

eval_df, eval_timing = timed_eval(
    finbert=finbert, model=trained_model, examples=test_data, use_amp=False
)

12/15/2025 14:21:35 - INFO - finbert.utils -   *** Example ***
12/15/2025 14:21:35 - INFO - finbert.utils -   guid: test-1
12/15/2025 14:21:35 - INFO - finbert.utils -   tokens: [CLS] the bristol port company has sealed a one million pound contract with cooper specialised handling to supply it with four 45 - ton ##ne , custom ##ised reach stack ##ers from ko ##ne ##cr ##ane ##s [SEP]
12/15/2025 14:21:35 - INFO - finbert.utils -   input_ids: 101 1996 7067 3417 2194 2038 10203 1037 2028 2454 9044 3206 2007 6201 17009 8304 2000 4425 2009 2007 2176 3429 1011 10228 2638 1010 7661 5084 3362 9991 2545 2013 12849 2638 26775 7231 2015 102 0 0 0 0 0 0 0 0 0 0
12/15/2025 14:21:35 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0
12/15/2025 14:21:35 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/15/2025 14:21:35 - IN

12/15/2025 14:21:35 - INFO - finbert.distillFinBert -     Num steps = 120
12/15/2025 14:21:35 - INFO - finbert.distillFinBert -   ***** Running evaluation ***** 
12/15/2025 14:21:36 - INFO - finbert.distillFinBert -     Num examples = 970
12/15/2025 14:21:36 - INFO - finbert.distillFinBert -     Batch size = 32
Testing: 100%|██████████| 31/31 [00:01<00:00, 25.37it/s]
12/15/2025 14:21:37 - INFO - finbert.utils -   *** Example ***
12/15/2025 14:21:37 - INFO - finbert.utils -   guid: test-1
12/15/2025 14:21:37 - INFO - finbert.utils -   tokens: [CLS] the bristol port company has sealed a one million pound contract with cooper specialised handling to supply it with four 45 - ton ##ne , custom ##ised reach stack ##ers from ko ##ne ##cr ##ane ##s [SEP]
12/15/2025 14:21:37 - INFO - finbert.utils -   input_ids: 101 1996 7067 3417 2194 2038 10203 1037 2028 2454 9044 3206 2007 6201 17009 8304 2000 4425 2009 2007 2176 3429 1011 10228 2638 1010 7661 5084 3362 9991 2545 2013 12849 2638 26775 7231 2

In [ ]:
def report(df, cols=['label','prediction','logits']):
    #print('Validation loss:{0:.2f}'.format(metrics['best_validation_loss']))
    cs = CrossEntropyLoss(weight=finbert.class_weights)
    loss = cs(torch.tensor(list(df[cols[2]])),torch.tensor(list(df[cols[0]])))
    print("Loss:{0:.2f}".format(loss))
    print("Accuracy:{0:.2f}".format((df[cols[0]] == df[cols[1]]).sum() / df.shape[0]) )
    print("\nClassification Report:")
    return_val = classification_report(df[cols[0]], df[cols[1]], output_dict=True)
    print(return_val)
    return_val["Loss"] = loss
    return_val["Accuracy"] = (df[cols[0]] == df[cols[1]]).sum() / df.shape[0]
    return return_val

In [ ]:
results['prediction'] = results.predictions.apply(lambda x: np.argmax(x,axis=0))

In [ ]:
wandb_report = report(results,cols=['labels','prediction','predictions'])

summary = {
        "device": str(finbert.device),
        "model_dir": str(cl_path),
        "train_wall_s": float(train_wall_s),
        "train_examples": int(len(train_data)),
        "train_examples_per_s": float((len(train_data) * finbert.config.num_train_epochs) / train_wall_s)
        if train_wall_s > 0
        else float("inf"),
        "model_size_mb": float(get_model_size_mb(trained_model)),
        "profile_train_steps": finbert.config.profile_train_steps,
        **(finbert.profile_results.get("training_summary", {}) or {}),
        **eval_timing,
        **wandb_report,
    }

wandb.log(summary)
wandb.finish()

/var/tmp/ipykernel_358109/3181304599.py:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  loss = cs(torch.tensor(list(df[cols[2]])),torch.tensor(list(df[cols[0]])))


Loss:0.41
Accuracy:0.81

Classification Report:
{'0': {'precision': 0.6987179487179487, 'recall': 0.8164794007490637, 'f1-score': 0.7530224525043178, 'support': 267.0}, '1': {'precision': 0.7090909090909091, 'recall': 0.9140625, 'f1-score': 0.7986348122866894, 'support': 128.0}, '2': {'precision': 0.9107505070993914, 'recall': 0.7808695652173913, 'f1-score': 0.8408239700374532, 'support': 575.0}, 'accuracy': 0.8082474226804124, 'macro avg': {'precision': 0.7728531216360831, 'recall': 0.8371371553221517, 'f1-score': 0.7974937449428201, 'support': 970.0}, 'weighted avg': {'precision': 0.8257761549004935, 'recall': 0.8082474226804124, 'f1-score': 0.8110886943947266, 'support': 970.0}}


Accuracy,▁
Loss,▁
accuracy,▁
eval_num_samples,▁
eval_samples_per_s,▁
eval_wall_s,▁
model_size_mb,▁
profile_train_steps,▁
train_backward_pass_ms,▁
train_data_transfer_ms,▁
+6,...
